In [11]:
from configuration import load_configuration

import utils

from os.path import join
import mne
from mne_bids import make_bids_basename, read_raw_bids
from mne_bids.utils import get_entity_vals

import plotting

import seaborn as sns

config = load_configuration()


nums = [100, 200, 300, 400, 500]
N = 10
peak_latency = 0.135


bids_root_path= "/media/marc/Medien/xmasoddballmatch-bids"
pipeline_name = "pipeline01"

ids = get_entity_vals(join(bids_root_path, "derivatives"), "sub") 

## Load Cleaned Raw Data from Disk

In [8]:
def load_and_epoch(id):
    # Load Data
    raw_filename = utils.get_derivative_file_name(
        config["bids_root_path"], id, config["pipeline_name"], ".fif", suffix="raw", processing="cleaned")
    events_filename = utils.get_derivative_file_name(
        config["bids_root_path"], id, config["pipeline_name"], ".txt", suffix="eve", processing="prepared")
    raw = mne.io.read_raw_fif(raw_filename, preload=True)
    events = mne.read_events(events_filename)

    # Epoch Data
    raw.pick(["eeg", "eog"])
    epochs = mne.Epochs(raw, events, config["events_dict"],
                        tmin=config["epoch_window"][0],
                        tmax=config["epoch_window"][1],
                        baseline=config["baseline_winow"],
                        #reject=config["diff_criterion"],
                        reject=None,
                        preload=True)

    # Fit AutoReject
    ar = AutoReject(verbose = False)
    ar.fit(epochs)

    return {"epochs":epochs, "events":events, "ar":ar}

subjects_data = {id: load_and_epoch(id) for id in ids}

Opening raw data file /media/marc/Medien/xmasoddballmatch-bids/derivatives/pipeline01/sub-001/sub-001_proc-cleaned_raw.fif...
    Range : 0 ... 2102271 =      0.000 ...  4105.998 secs
Ready.
Reading 0 ... 2102271  =      0.000 ...  4105.998 secs...
Not setting metadata
Not setting metadata
39842 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 39842 events and 309 original time points ...


KeyboardInterrupt: 

## Epoch Data

## Fit AutoReject

In [6]:
cond1 = "random/standard"
cond2 = "random/deviant"

def get_mean_amplitudes(evokeds, window, picks = "all"):

    means = []
    if isinstance(evokeds, list):
        for i, evoked in enumerate(evokeds):
            evoked = evoked.copy().pick(picks)

            _window = np.arange(evoked.time_as_index(window[0]),
                                evoked.time_as_index(window[1]))

            data = evoked.data[:, _window]
            mean = data.mean()
            means.append(mean)
    else:
        return get_mean_amplitudes([evokeds], window, picks)[0]

    return means


p_values = {num:[None] * N for num in nums}
e_values = {num:[None] * N for num in nums}

for i, num in enumerate(nums):
    for n in N:

        for i,id in enumerate(ids):
            # get epochs
            epochs = subjects_data[id]["epochs"]
            evokeds = {cond: [] for cond in [cond1, cond2]}
            
            for cond in [cond1, cond2]:
                # select epochs
                s_epochs = epochs[cond]

                # draw a random sample of epochs
                idx = list(range(len(s_epochs)))
                retain_idx = np.random.choice(idx, num)
                drop_idx = set(idx) - set(retain_idx)
                subsample_epochs = s_epochs.copy().drop(drop_idx)

                # apply autoreject
                subsample_epochs = ar.transform(subsample_epochs)
                # average
                evokeds[cond].append(subsample_epochs.average())

            # calculate amplitude differenc (effect estimate)
            diff_waves = [mne.combine_evoked([e1,e2], [1,-1]) for e1,e2 in zip(evokeds["random/standard"], evokeds["random/deviant"])]
            mean_amplitudes = get_mean_amplitudes(diff_waves, peakwindow, picks = ["FZ"]) 
            e_values[num] = mean(mean_amplitudes)
            # calculate p-value
            # store value somewhere

TypeError: 'int' object is not iterable